In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 3500) #allows 3500 rows to be displayed
pd.options.mode.chained_assignment = None #supresses 'set with copy' warnings

In [2]:
# run mode 'spy' analyzes all companies from the S&P 500
# run mode 'wilshire' analyzes all companies from the Wilshire 5000
run_mode = 'spy'
#run_mode = 'wilshire'

In [3]:
# Get our set of companies to screen
if run_mode == 'spy':
    # S&P 500 companies are extracted from wikipedia
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    df = df.rename(columns = {'Symbol': 'Ticker', 
                              'Security': 'Company'})
    df = df [['Ticker', 'Company', 'GICS Sector', 'GICS Sub-Industry']]
elif run_mode == 'wilshire':
    # Wilshire 5000 tickers are taken from a local file
    df = pd.read_excel('wilshire_5000_stocks.xlsx', header=3)
    df['Ticker'] = df['AAC']
else:
    print(f"ERROR: used invalid run mode: '{run_mode}'")

df

,Ticker,Company,GICS Sector,GICS Sub-Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Biotechnology
4,ACN,Accenture,Information Technology,IT Consulting & Other Services
5,ADBE,Adobe Inc.,Information Technology,Application Software
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders
8,AFL,Aflac,Financials,Life & Health Insurance
9,A,Agilent Technologies,Health Care,Life Sciences Tools & Services


In [8]:
# Filter out Financials and Utilities
if run_mode == 'spy':
    def sector_filter(sector):
        if sector in ('Financials', 'Utilities'):
            return False
        else:
            return True

    initial_row_count = df.shape[0]
    df = df[df['GICS Sector'].apply(sector_filter)]
    new_row_count = df.shape[0]
    print(f'''Began with {initial_row_count} rows, {new_row_count} remain after filtering out companies from the financials and utilities sectors.''')

Began with 503 rows, 401 remain after filtering out companies from the financials and utilities sectors.


In [31]:
import yfinance as yf # Yahoo Finance API
from time import sleep
from joblib import Parallel, delayed # parallelizes API-calls
from tqdm import tqdm # provides progress bar

def get_ticker_info(ticker):
    # Calls Yahoo Finance API to get financial data for a given ticker
    # Includes 3x retry logic to deal with unreliable API behavior
    attempts = 0
    while attempts < 3:
        try:
            return yf.Ticker(ticker).info
        except Exception as err:
            attempts = attempts + 1
            print(f"WARNING: API call failed for ticker {ticker}... with error \n{err}\n Trying again in 3 seconds.")
            sleep(3)
    print(f"WARNING: Could not get API response for ticker {ticker} after 3 attempts... skipping!")
    return np.nan

# Executes get_ticker_info for all companies in our dataframe
# Parallelizes over n cores
n_cores = 4
ticker_info_array = Parallel(n_jobs=n_cores)(
    delayed(get_ticker_info)(t) 
    for t in tqdm(df['Ticker'].values, 
                  total=df.shape[0], 
                  position=0, 
                  leave=True))

100%|██████████| 503/503 [00:22<00:00, 22.37it/s]


In [32]:
# Extract financial metrics of interest
def get_symbol(ticker_info):
    # Extracts sector from ticker info
    if 'shortName' in ticker_info:
        return ticker_info['shortName']
    return np.nan

def get_return_on_assets(ticker_info):
    # Extracts return-on-assets metric from ticker info
    if 'returnOnAssets' in ticker_info:
        return ticker_info['returnOnAssets']
    return np.nan

def get_price_to_earnings(ticker_info):
    # Extracts price-to-earnings ratio from ticker info
    if 'trailingPE' in ticker_info:
        return ticker_info['trailingPE']
    if 'forwardPE' in ticker_info:
        return ticker_info['forwardPE']
    return np.nan

def get_sector(ticker_info):
    # Extracts sector from ticker info
    if 'sector' in ticker_info:
        return ticker_info['sector']
    return np.nan

financials = pd.DataFrame(
    [(get_symbol(ticker_info), 
      get_return_on_assets(ticker_info),
      get_price_to_earnings(ticker_info),
      get_sector(ticker_info))
     for ticker_info in ticker_info_array if ticker_info],
    columns = ['Ticker', 'ROA', 'PE', 'Sector'])

financials

,Ticker,ROA,PE,Sector
0,3M Company,0.07580,9.032352,Industrials
1,A.O. Smith Corporation,0.14148,22.395664,Industrials
2,Abbott Laboratories,0.06112,36.631900,Healthcare
3,AbbVie Inc.,0.08071,65.512870,Healthcare
4,Accenture plc,0.12973,34.801110,Technology
5,Adobe Inc.,0.14598,46.700510,Technology
6,"Advanced Micro Devices, Inc.",0.00370,330.234740,Technology
7,The AES Corporation,0.03362,44.661766,Utilities
8,AFLAC Incorporated,0.02639,10.350900,Financial Services
9,"Agilent Technologies, Inc.",0.07797,32.708830,Healthcare


In [35]:
df = financials.copy(deep=True)
df['PE'] = df['PE'].astype('float64')
df = df[(df['PE'].notnull()) &
        (df['ROA'].notnull()) &
        (df['PE'] > 0) &
        (df['ROA'] > 0)]
df['PE'] = pd.to_numeric(df['PE'])
df['ROA'] = pd.to_numeric(df['ROA'])
df

,Ticker,ROA,PE,Sector
0,3M Company,0.07580,9.032352,Industrials
1,A.O. Smith Corporation,0.14148,22.395664,Industrials
2,Abbott Laboratories,0.06112,36.631900,Healthcare
3,AbbVie Inc.,0.08071,65.512870,Healthcare
4,Accenture plc,0.12973,34.801110,Technology
5,Adobe Inc.,0.14598,46.700510,Technology
6,"Advanced Micro Devices, Inc.",0.00370,330.234740,Technology
7,The AES Corporation,0.03362,44.661766,Utilities
8,AFLAC Incorporated,0.02639,10.350900,Financial Services
9,"Agilent Technologies, Inc.",0.07797,32.708830,Healthcare


In [36]:
df['PE'] = df['PE'].astype('float64')
df = df[(df['PE'].notnull()) &
        (df['ROA'].notnull()) &
        (df['PE'] > 0) &
        (df['ROA'] > 0)]
df['PE'] = pd.to_numeric(df['PE'])
df['ROA'] = pd.to_numeric(df['ROA'])

In [37]:
# Final score is based on relative rankings for price-to-earnings and return-on-assets metrics (equally weighted)
# Companies with a relatively low price-to-earnings and high return-on-assets will come to the top
df['ROA_rank'] = df['ROA'].rank(ascending=False)
df['PE_rank'] = df['PE'].rank(ascending=True) 
df['Score'] = (df['ROA_rank'] + df['PE_rank']).rank(ascending=True)
df = df.sort_values('Score')

In [38]:
df.iloc[0:100]

,Ticker,ROA,PE,Sector,ROA_rank,PE_rank,Score
38,APA Corporation,0.15080,3.270811,Energy,30.0,1.0,1.0
21,"Altria Group, Inc.",0.19967,8.934355,Consumer Defensive,12.0,26.0,2.0
172,"EOG Resources, Inc.",0.13993,8.841538,Energy,44.0,21.0,3.0
145,Devon Energy Corporation,0.12971,7.592845,Energy,61.0,12.0,4.0
387,"PulteGroup, Inc.",0.13867,9.087030,Consumer Cyclical,46.0,30.0,5.5
464,Valero Energy Corporation,0.12053,5.672151,Energy,72.0,4.0,5.5
425,"Steel Dynamics, Inc.",0.13551,9.045082,Basic Materials,52.0,28.0,7.0
156,"D.R. Horton, Inc.",0.12164,10.490988,Consumer Cyclical,70.0,47.0,8.0
347,Nucor Corporation,0.11483,10.736667,Basic Materials,78.0,50.0,9.0
99,"CF Industries Holdings, Inc.",0.10216,10.161372,Basic Materials,96.0,42.0,10.0


In [ ]:
df.sort_values("PE")

In [ ]:
df[df['Ticker'] == 'HPQ']

In [ ]:
df.to_csv('yfinance.csv')